In [ ]:
import os
import subprocess
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from mygene import MyGeneInfo

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

from sklearn.decomposition import PCA

from pathlib import Path

In [ ]:
WORK_DIR = Path.cwd().absolute()
WORK_DIR

In [ ]:
SALMON_IMAGE = "combinelab/salmon:1.10.0"

In [ ]:
# SAMPLES = [
#     "SRR16243477", "SRR16243498", "SRR16243497",  # Control
#     "SRR16243483", "SRR16243482", "SRR16243481"   # Treatment
# ]

SAMPLES = [
    "SRR13416176", "SRR13416177", "SRR13416178",  # CAMT-1
    "SRR13416172", "SRR13416173", "SRR13416174"   # WT
]

Download a reference transcriptome for **C.elegans**

In [ ]:
C_ELEGANS_URL = "https://hgdownload.soe.ucsc.edu/gbdb/ce11/ncbiRefSeq/seqNcbiRefSeq.rna.fa"

In [ ]:
fa_path = WORK_DIR / "c_elegans.fa"

if not fa_path.exists():
    print("Downloading reference transcriptome ...")
    r = requests.get(url, stream=True)

    with open(fa_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=256*1024):
            f.write(chunk)

Creating Salmon index

In [ ]:
S_INDEX = "salmon_index"

if not (WORK_DIR / S_INDEX).exists():
    print("Creating Salmon index ...")

    subprocess.run([
        "docker", "run", "--rm",
        "-v", f"{WORK_DIR}:/data",
        SALMON_IMAGE,
        "salmon", "index",
        "-t", "/data/c_elegans.fa",
        "-i", "/data/salmon_index"
    ], check=True)

Download the RNA-Seq reads

In [ ]:
for sra_id in SAMPLES:
    sra_file = WORK_DIR / f"{sra_id}.sra"
    if not sra_file.exists():
        print(f"Скачиваем {sra_id}...")
        subprocess.run(["prefetch", sra_id], check=True)

Converting .sra to .fastq

In [ ]:
for sra_id in SAMPLES:
    print(f"Конвертируем {sra_id} в FASTQ...")
    subprocess.run([
        "docker", "run", "--rm",
        "-v", f"{WORK_DIR}:/data",
        "ncbi/sra-tools:3.0.0",
        "fasterq-dump",
        f"/data/{sra_id}",
        "-O", "/data",
        "--force",
        "--threads", "32"
    ], check=True)

Quantify transcript expression with `salmon quant`

In [ ]:
for fq in WORK_DIR.glob("*.fastq"):
    print(f"Обработка {fq.name}...")
    subprocess.run([
        "docker", "run", "--rm",
        "-v", f"{WORK_DIR}:/data",
        SALMON_IMAGE,
        "salmon", "quant",
        "-i", "/data/salmon_index",
        "-l", "A",
        "-r", f"/data/{fq.name}",
        "--validateMappings",
        "-o", f"/data/quants/{fq.stem}"
    ], check=True)

Create a metadata dataframe to label the RNA-Seq `quant.sf` files

In [ ]:
counts = pd.DataFrame()

# Загружаем данные для каждого образца
for sample in SAMPLES:
    quant_file = WORK_DIR / "quants" / sample / "quant.sf"
    df = pd.read_csv(quant_file, sep="\t", usecols=["Name", "NumReads"])
    counts[sample] = df.set_index("Name")["NumReads"]

In [ ]:
counts = counts.fillna(0).astype(int).T
counts

Differential Expression Analysis

In [ ]:
metadata = pd.DataFrame({
    "condition": ["Control"]*3 + ["Treatment"]*3
}, index=SAMPLES)

metadata

In [ ]:
dds = DeseqDataSet(
    counts=counts,
    metadata=metadata,
    design_factors=["condition"],
    n_cpus=32
)

In [ ]:
dds.deseq2()

In [ ]:
stat_res = DeseqStats(dds, contrast=["condition", "Control", "Treatment"])
stat_res.summary()
results_df = stat_res.results_df

Gene annotation

In [ ]:
mg = MyGeneInfo()
genes = results_df.index.str.split('.').str[0].tolist()
gene_info = mg.querymany(genes, scopes='ensembl.gene', fields='symbol', species=6239)

In [ ]:
gene_dict = {}
for info in gene_info:
    gene_dict[info['query']] = info.get('symbol', info['query'])

In [ ]:
# gene_dict

In [ ]:
results_df['symbol'] = results_df.index.map(lambda x: gene_dict.get(x.split('.')[0], x))

In [ ]:
results_df

In [ ]:
# plt.style.use('seaborn')
# matplotlib.rcParams['figure.dpi'] = 150

In [ ]:
results_df.columns

In [ ]:
# Создаем копию DataFrame, чтобы избежать изменений в оригинале
df = stat_res.results_df.copy()

# Логарифмируем p-value
df["-log10(pvalue)"] = -np.log10(df["pvalue"])

# Определяем значимость
df["significance"] = "NS"  # Default: незначимые гены
df.loc[(df["pvalue"] < 1e-6) & (abs(df["log2FoldChange"]) > 1), "significance"] = "p-value and log2 FC"
df.loc[(df["pvalue"] < 1e-6) & (abs(df["log2FoldChange"]) <= 1), "significance"] = "p-value"
df.loc[(df["pvalue"] >= 1e-6) & (abs(df["log2FoldChange"]) > 1), "significance"] = "Log2 FC"

# Определяем цвета как в R
colors = {
    "NS": "gray",
    "Log2 FC": "blue",
    "p-value": "purple",
    "p-value and log2 FC": "red"
}

# Создаем фигуру
plt.figure(figsize=(9, 9))
sns.scatterplot(
    x=df["log2FoldChange"],
    y=df["-log10(pvalue)"],
    hue=df["significance"],
    palette=colors,
    edgecolor=None,
    alpha=0.7
)

# Добавляем вертикальные и горизонтальную линии
plt.axhline(y=-np.log10(1e-6), color="black", linestyle="--")
plt.axvline(x=-1, color="black", linestyle="--")
plt.axvline(x=1, color="black", linestyle="--")

# Добавляем подписи для самых значимых генов
top_genes = df[df["significance"] == "p-value and log2 FC"].nlargest(30, "-log10(pvalue)")
for i, row in top_genes.iterrows():
    plt.text(row["log2FoldChange"], row["-log10(pvalue)"], row.name,
             fontsize=8, ha="right" if row["log2FoldChange"] < 0 else "left")

# Подписи осей
plt.xlabel(r"$Log_2$ fold change")
plt.ylabel(r"$-Log_{10} P$")
plt.title("Volcano plot")

# Легенда
plt.legend(title="Significance", loc="upper right")

# Сохранение графика
plt.savefig("volcanoPlot.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Create a copy to avoid modifying the original dataframe
df = stat_res.results_df.copy()

# Compute -log10(p-value)
df["-log10(pvalue)"] = -np.log10(df["pvalue"])

# Define significance categories
df["significance"] = "Not Significant"  # Default
df.loc[(df["pvalue"] < 1e-6) & (abs(df["log2FoldChange"]) > 1), "significance"] = "Significant (p-value & log2 FC)"
df.loc[(df["pvalue"] < 1e-6) & (abs(df["log2FoldChange"]) <= 1), "significance"] = "Significant (p-value)"
df.loc[(df["pvalue"] >= 1e-6) & (abs(df["log2FoldChange"]) > 1), "significance"] = "Significant (log2 FC)"

# Define colors for categories
colors = {
    "Not Significant": "lightgray",
    "Significant (log2 FC)": "royalblue",
    "Significant (p-value)": "mediumorchid",
    "Significant (p-value & log2 FC)": "crimson"
}

# Create figure
plt.figure(figsize=(10, 10))

# Scatter plot
sns.scatterplot(
    x=df["log2FoldChange"],
    y=df["-log10(pvalue)"],
    hue=df["significance"],
    palette=colors,
    edgecolor="black",
    alpha=0.75,  # Transparency for better visualization
    s=40  # Marker size
)

# Add threshold lines
plt.axhline(y=-np.log10(1e-6), color="black", linestyle="--", linewidth=1)
plt.axvline(x=-1, color="black", linestyle="--", linewidth=1)
plt.axvline(x=1, color="black", linestyle="--", linewidth=1)

# Label the top 20 most significant genes
top_genes = df[df["significance"] == "Significant (p-value & log2 FC)"].nlargest(20, "-log10(pvalue)")
for i, row in top_genes.iterrows():
    plt.text(
        row["log2FoldChange"], row["-log10(pvalue)"], row.name,
        fontsize=9, fontweight='bold',
        ha="right" if row["log2FoldChange"] < 0 else "left",
        bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.3")
    )

# Labels and title
plt.xlabel(r"$\log_2$ Fold Change", fontsize=14, fontweight="bold")
plt.ylabel(r"$-\log_{10}$ p-value", fontsize=14, fontweight="bold")
plt.title("Volcano Plot: Treatment vs. Control", fontsize=16, fontweight="bold")

# Legend adjustments
plt.legend(title="Significance", loc="upper right", fontsize=12, frameon=True, fancybox=True)

# Save and show plot
plt.savefig("volcanoPlot_scientific.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import pydeseq2
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Предположим, что `dds` уже создан как объект DeseqDataSet

# 1. Обязательные шаги для VST
dds.vst_fit()  # Заменяет fit_dispersion_trend() и fit_genewise_dispersions()

# 2. Применяем VST преобразование
dst_counts = dds.vst_transform(counts=None)

# Остальные шаги остаются без изменений
dst = pd.DataFrame(dst_counts)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(dst)

condition = dds.obs['condition'].values
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['condition'] = condition

plt.figure(figsize=(7, 5))
for condition_value in pca_df['condition'].unique():
    subset = pca_df[pca_df['condition'] == condition_value]
    plt.scatter(subset['PC1'], subset['PC2'], label=condition_value)

plt.title('PCA of Variance Stabilized Counts')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend(title='Condition')
plt.grid(True)
plt.show()

Heatmap

In [ ]:
conditions = ["Control"] * 3 + ["Treatment"] * 3

In [ ]:
# Выбор топ-50 генов
top_genes = results_df.sort_values('pvalue').head(50).index

# Z-score нормализация
normalized_data = counts.T.loc[top_genes].apply(
    lambda x: (x - x.mean())/x.std(), 
    axis=1
)

# Создание теплокарты
plt.figure(figsize=(12, 8))
sns.heatmap(
    normalized_data,
    cmap="coolwarm",
    xticklabels=conditions,
    yticklabels=results_df.loc[top_genes, 'symbol']
)

plt.title("Top 50 Differentially Expressed Genes")
plt.xlabel("Sample")
plt.ylabel("Gene")
plt.savefig("heatmap.png", bbox_inches='tight')
plt.show()